In [ ]:
# Define URLs
h5_url <- "https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSM7777523&format=file&file=GSM7777523%5FD1%5Ffiltered%5Ffeature%5Fbc%5Fmatrix%2Eh5"
tar_gz_url <- "https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSM7777523&format=file&file=GSM7777523%5FD1%5Fspatial%2Etar%2Egz"

# Define file names
h5_file <- "D1_filtered_feature_bc_matrix.h5"
tar_gz_file <- "D1_spatial.tar.gz"

# Download files
download.file(h5_url, destfile = h5_file, mode = "wb")  # mode = "wb" ensures binary download
download.file(tar_gz_url, destfile = tar_gz_file, mode = "wb")

In [ ]:
untar(tar_gz_file, exdir = "D1_spatial")

In [ ]:
dir.create("D1", showWarnings = FALSE)


In [ ]:
file.rename("/kaggle/working/D1_filtered_feature_bc_matrix.h5", "D1/D1_filtered_feature_bc_matrix.h5")
file.rename("D1_spatial", "D1/D1_spatial")


In [ ]:
file.remove('/kaggle/working/D1_spatial.tar.gz')

In [1]:
custom_lib <- "/kaggle/working/KO"  
default_lib <- .libPaths()[1] 
#
packages <- list.dirs(custom_lib, full.names = TRUE, recursive = FALSE)

# move packages from custom to default library path
for (pkg in packages) {
  pkg_name <- basename(pkg)  
  target_path <- file.path(default_lib, pkg_name)
  
  # check if the package already exists in the default path
  if (!dir.exists(target_path)) {
    file.copy(pkg, default_lib, recursive = TRUE)
  } else {
    message(paste("package already exists in default library:", pkg_name))
  }
}
print('-----------------All packages are moved----------------------')

[1] "-----------------All packages are moved----------------------"


In [2]:
custom_lib <- "/kaggle/input/library-data/my_r_libs"  
default_lib <- .libPaths()[1] 

packages <- list.dirs(custom_lib, full.names = TRUE, recursive = FALSE)

# move packages from custom to default library path
for (pkg in packages) {
  pkg_name <- basename(pkg)  
  target_path <- file.path(default_lib, pkg_name)
  
  # check if the package already exists in the default path
  if (!dir.exists(target_path)) {
    file.copy(pkg, default_lib, recursive = TRUE)
  } else {
    message(paste("package already exists in default library:", pkg_name))
  }
}
print('-----------------All packages are moved----------------------')

package already exists in default library: bslib

package already exists in default library: digest

package already exists in default library: e1071

package already exists in default library: evaluate

package already exists in default library: knitr

package already exists in default library: KODAMA

package already exists in default library: KODAMAextra

package already exists in default library: openssl

package already exists in default library: Rcpp

package already exists in default library: RcppArmadillo

package already exists in default library: RcppEigen

package already exists in default library: reticulate

package already exists in default library: rmarkdown

package already exists in default library: RSpectra

package already exists in default library: tinytex

package already exists in default library: uuid

package already exists in default library: withr

package already exists in default library: xfun

package already exists in default library: yaml



[1] "-----------------All packages are moved----------------------"


In [ ]:
# BiocManager::install('STexampleData')
# library(STexampleData)
library(ggspavis)
library(ggplot2)
library(scater)
library(scran)
library(igraph)
library(pheatmap)
library(ggExtra)
library(nnSVG)
library(scater)
library(scran)
library(scry)
library(SPARK)
library(harmony)
library(Seurat)
library(spatialLIBD)
library(KODAMAextra)
library(mclust)
library(slingshot)
library(irlba)
library(dplyr)
library(hdf5r)
library(Matrix)
library(Seurat)
library(ggplot2)
library(patchwork)
library(dplyr)


In [ ]:
library(Seurat)
library(KODAMA)
library(KODAMAextra)
library(SPARK)
install.packages('Rnanoflann')

In [ ]:
# making all figures bigger
options(repr.plot.width = 15, repr.plot.height = 10)

In [ ]:
clusters_colors=c('#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b',
       '#e377c2', '#7f7f7f', '#bcbd22', '#17becf')

In [ ]:
files <- c("A1", "B1", "C1", "D1")  # List of tissue names
st <- list()  # Empty list to store Seurat objects


In [ ]:
for (file in files) {
  # Read count matrix
  matrix_path <- paste0("/kaggle/working/", file, "/", file, "_filtered_feature_bc_matrix.h5")
  matrix <- Read10X_h5(matrix_path)
  
  # Read spatial image
  img_path <- paste0("/kaggle/working/", file, "/", "spatial")
  img <- Read10X_Image(img_path)
  
  # Create Seurat object
  seurat_obj <- CreateSeuratObject(counts = matrix, assay = "Spatial")
  
  # Add spatial image to Seurat object
  seurat_obj@images$spatial <- img
  
  # Add tissue metadata
  seurat_obj$Tissue <- file
  
  # Store in list
  st[[file]] <- seurat_obj
}


In [ ]:
st$A1

In [ ]:
head(st$A1@meta.data)

# Qc

In [ ]:
for (tissue_name in names(st)) {
    data <- st[[tissue_name]]

    data$log10GenesPerUMI <- log10(data$nFeature_Spatial) / log10(data$nCount_Spatial)

    data$mitoRatio <- PercentageFeatureSet(object = data, pattern = "^MT-")
    
    data$mitoRatio <- data@meta.data$mitoRatio / 100

    st[[tissue_name]] <- data
}


In [ ]:
head(st$B1)

In [ ]:
SpatialFeaturePlot(st$A1, features=c('nCount_Spatial','nFeature_Spatial','log10GenesPerUMI'))

In [ ]:
SpatialFeaturePlot(st$B1, features=c('nCount_Spatial','nFeature_Spatial','log10GenesPerUMI'))

In [ ]:
SpatialFeaturePlot(st$C1, features=c('nCount_Spatial','nFeature_Spatial','log10GenesPerUMI'))

In [ ]:
SpatialFeaturePlot(st$D1, features=c('nCount_Spatial','nFeature_Spatial','log10GenesPerUMI'))

# print mitc precentage 
# filter genes based on mito genes

In [ ]:
gene_sums <- rowSums(st$A1@assays$Spatial$counts > 0)  # Sum of gene counts across spots

gene_sums_df <- data.frame(Gene = names(gene_sums), Total_Counts = gene_sums)

ggplot(gene_sums_df, aes(x = Total_Counts)) +
  geom_density(fill = "blue", alpha = 0.4) +
  scale_x_log10() +  # Log scale for better visualization
  theme_minimal() +
  labs(title = "Density Plot of Gene Expression", x = "Total Counts per Gene (log10)", y = "Density")+
  geom_vline(xintercept = 3, color = "red", linetype = "dashed", size = 1)


In [ ]:
gene_sums <- rowSums(st$B1@assays$Spatial$counts > 0)  # Sum of gene counts across spots

gene_sums_df <- data.frame(Gene = names(gene_sums), Total_Counts = gene_sums)

ggplot(gene_sums_df, aes(x = Total_Counts)) +
  geom_density(fill = "blue", alpha = 0.4) +
  scale_x_log10() +  # Log scale for better visualization
  theme_minimal() +
  labs(title = "Density Plot of Gene Expression", x = "Total Counts per Gene (log10)", y = "Density")+
  geom_vline(xintercept = 3, color = "red", linetype = "dashed", size = 1)


In [ ]:
gene_sums <- rowSums(st$C1@assays$Spatial$counts > 0)  # Sum of gene counts across spots

gene_sums_df <- data.frame(Gene = names(gene_sums), Total_Counts = gene_sums)

ggplot(gene_sums_df, aes(x = Total_Counts)) +
  geom_density(fill = "blue", alpha = 0.4) +
  scale_x_log10() +  # Log scale for better visualization
  theme_minimal() +
  labs(title = "Density Plot of Gene Expression", x = "Total Counts per Gene (log10)", y = "Density")+
  geom_vline(xintercept = 3, color = "red", linetype = "dashed", size = 1)


In [ ]:
gene_sums <- rowSums(st$D1@assays$Spatial$counts > 0)  # Sum of gene counts across spots

gene_sums_df <- data.frame(Gene = names(gene_sums), Total_Counts = gene_sums)

ggplot(gene_sums_df, aes(x = Total_Counts)) +
  geom_density(fill = "blue", alpha = 0.4) +
  scale_x_log10() +  # Log scale for better visualization
  theme_minimal() +
  labs(title = "Density Plot of Gene Expression", x = "Total Counts per Gene (log10)", y = "Density")+
  geom_vline(xintercept = 3, color = "red", linetype = "dashed", size = 1)


In [ ]:
dim(st$A1)

In [ ]:
min_spots <- 3  # Removing genes expressed in fewer than 3 spots
for (tissue_name in names(st)) {
    st[[tissue_name]] <- subset(st[[tissue_name]], 
                                features = rownames(st[[tissue_name]])[Matrix::rowSums(st[[tissue_name]]@assays$Spatial$counts > 0) > min_spots])
}


In [ ]:
dim(st$A1)
dim(st$B1)
dim(st$C1)
dim(st$D1)
# removed genes

## mitochondrial genes weren't totaly removed as it didn't affect the clustering badly it impoved it

In [ ]:
for (tissue_name in names(st)){
    st[[tissue_name]]= SCTransform(st[[tissue_name]], assay ='Spatial')
}
# output of this cell was removed to decrease notebook storage 

In [ ]:
names(st$A1@assays)

In [ ]:
for ( tissue in names(st)){
    st[[tissue]] <- FindVariableFeatures(st[[tissue]])
    }

In [ ]:
for ( tissue in names(st)){
    # st[[tissue]] <- ScaleData(st[[tissue]])
    st[[tissue]] <- RunPCA(st[[tissue]])
    }


In [ ]:
n.cores=40
splitting = 100
spatial.resolution = 0.3
aa_noise=3
gene_number=2000
graph = 20
seed=123

In [ ]:
for (tissue in names(st)){
    st[[tissue]] <- RunKODAMAmatrix(st[[tissue]],
                           FUN = "fastpls",
                           landmarks = 100000,
                           splitting = 300,
                           ncomp=50
                           )
    saveRDS(st[[tissue]], file = paste0(tissue, '.rds'))
    }
# output was removed 

In [ ]:
files <- c("A1", "B1", "C1", "D1")  # List of tissue names
st <- list()  # Empty list to store Seurat objects
for (file in files) {
    seurat_obj=readRDS(paste0(file,'.rds'))
  # Store in list
  st[[file]] <- seurat_obj
}


In [ ]:
library(bluster)
library(igraph)
library(KODAMA)
library(KODAMAextra)
library(Seurat)
library(ggplot2)

In [ ]:
config=umap.defaults
config$n_sgd_threads = "auto"
for (tissue_name in names(st)) {
    st[[tissue_name]] <- RunKODAMAvisualization(st[[tissue_name]],method="UMAP",config=config)
}

In [ ]:
kodama_dims <- Embeddings(st[['A1']], reduction='KODAMA')

In [ ]:
snn_graph <- bluster::makeSNNGraph(kodama_dims, k = 30) 

In [ ]:
snn_graph

In [ ]:
clusters <- igraph::cluster_walktrap(snn_graph)


cluster_labels <- as.character(membership(clusters))

In [ ]:
unique(cluster_labels)

In [ ]:
st[['A1']]@meta.data$clusters <- cluster_labels


In [ ]:
df=data.frame(kodama_dims)
df['KODAMA']=st[['A1']]@meta.data$clusters

In [ ]:
head(df)

In [ ]:
ggplot(df, aes(Dimensions_1, Dimensions_2, color = KODAMA)) +labs(title="kodama") +
  geom_point(size = 2) +
  theme_bw() + theme(legend.position = "bottom")+
  guides(color = guide_legend(nrow = 2, 
                              override.aes = list(size = 2)))

In [ ]:
DimPlot(st[['A1']],reduction='KODAMA' ,c(1, 2), label =TRUE ,group.by='clusters')

In [ ]:
SpatialDimPlot(st[['A1']], group.by='clusters',pt.size.factor=2)

In [ ]:
library(ggplot2)

p <- SpatialDimPlot(st[['A1']], group.by='clusters',pt.size.factor=2, label= TRUE)

ggsave("A1.png", plot=p, width=8, height=6, dpi=300)


# patient 1 av ethe highest degree of cancer ⅢC/T2N3aM0ⅢC/T2N3aM0

# B1 clustering

In [ ]:
kodama_dims <- Embeddings(st[['B1']], reduction='KODAMA')

In [ ]:
snn_graph <- bluster::makeSNNGraph(kodama_dims, k = 40) 

In [ ]:
snn_graph

In [ ]:
clusters <- igraph::cluster_walktrap(snn_graph)


cluster_labels <- as.character(membership(clusters))
unique(cluster_labels)

In [ ]:
st[['B1']]@meta.data$clusters <- cluster_labels
df=data.frame(kodama_dims)
df['KODAMA']=st[['B1']]@meta.data$clusters

In [ ]:
ggplot(df, aes(Dimensions_1, Dimensions_2, color = KODAMA)) +labs(title="kodama") +
  geom_point(size = 2) +
  theme_bw() + theme(legend.position = "bottom")+
  guides(color = guide_legend(nrow = 2, 
                              override.aes = list(size = 2)))

In [ ]:
DimPlot(st[['B1']],reduction='KODAMA' ,c(1, 2), label =TRUE ,group.by='clusters')

In [ ]:
p <- SpatialDimPlot(st[['B1']], group.by='clusters',pt.size.factor=2,label =TRUE)

ggsave("B1.png", plot=p, width=8, height=6, dpi=300)

# C1 clustering

In [ ]:
kodama_dims <- Embeddings(st[['C1']], reduction='KODAMA')

In [ ]:
snn_graph <- bluster::makeSNNGraph(kodama_dims, k = 30) 

In [ ]:
snn_graph

In [ ]:
clusters <- igraph::cluster_walktrap(snn_graph)


cluster_labels <- as.character(membership(clusters))
unique(cluster_labels)

In [ ]:
st[['C1']]@meta.data$clusters <- cluster_labels
df=data.frame(kodama_dims)
df['KODAMA']=st[['C1']]@meta.data$clusters

In [ ]:
ggplot(df, aes(Dimensions_1, Dimensions_2, color = KODAMA)) +labs(title="kodama") +
  geom_point(size = 2) +
  theme_bw() + theme(legend.position = "bottom")+
  guides(color = guide_legend(nrow = 2, 
                              override.aes = list(size = 2)))

In [ ]:
p <- SpatialDimPlot(st[['C1']], group.by='clusters',pt.size.factor=2,label =TRUE)


ggsave("C1.png", plot=p, width=8, height=6, dpi=300)

# D1 clustering

In [ ]:
kodama_dims <- Embeddings(st[['D1']], reduction='KODAMA')

In [ ]:
snn_graph <- bluster::makeSNNGraph(kodama_dims, k = 30) 

In [ ]:
snn_graph

In [ ]:
clusters <- igraph::cluster_walktrap(snn_graph)


cluster_labels <- as.character(membership(clusters))
unique(cluster_labels)

In [ ]:
st[['D1']]@meta.data$clusters <- cluster_labels
df=data.frame(kodama_dims)
df['KODAMA']=st[['D1']]@meta.data$clusters

In [ ]:
ggplot(df, aes(Dimensions_1, Dimensions_2, color = KODAMA)) +labs(title="kodama") +
  geom_point(size = 2) +
  theme_bw() + theme(legend.position = "bottom")+
  guides(color = guide_legend(nrow = 2, 
                              override.aes = list(size = 2)))

In [ ]:
p <- SpatialDimPlot(st[['D1']], group.by='clusters',pt.size.factor=2,label =TRUE)



ggsave("D1.png", plot=p, width=8, height=6, dpi=300)

In [ ]:
files <- c("A1", "B1", "C1", "D1")  # List of tissue names
st <- list()  # Empty list to store Seurat objects
for (file in files) {
    seurat_obj=readRDS(paste0(file,'.rds'))
  # Store in list
  st[[file]] <- seurat_obj
}


In [ ]:
FindAllMarkers

In [ ]:
library(msigdbr)
immune_genes_df <- msigdbr(species = "Homo sapiens", category = "C7")  # C7: Immunologic Signature Genes
immune_genes <- unique(immune_genes_df$gene_symbol)

write.csv(immune_genes, "immune_gene_list.csv", row.names = FALSE)


In [ ]:
immune_genes =read.csv('immune_gene_list.csv')

In [ ]:
st[['A1']]$Immune_Score <- PercentageFeatureSet(st[['A1']], features = immune_genes)

In [ ]:
dadadaf

In [ ]:

for (cluster in unique(st@meta.data$cluster)) {
    markers <- FindMarkers(st, 
                           ident.1 = cluster, 
                           group.by = "clusters",
                           only.pos = TRUE, 
                           min.pct = 0.25, 
                           logfc.threshold = 0.25)
    
    # Filter markers to keep only immune genes
    immune_markers <- markers[rownames(markers) %in% immune_genes, ]
    
    write.csv(immune_markers, file = paste0("Cluster_", cluster, "_immune_genes.csv"), row.names = TRUE)
}


In [ ]:
st[['A1']]$Immune_Score <- PercentageFeatureSet(object = st[['A1']], pattern = "^IG")


In [ ]:
library(dplyr)
immune_percentage <- stst[['A1']]@meta.data %>%
  group_by(Tissue, cluster) %>%
  summarise(immune_cell_percent = mean(Immune_Score))  # Average immune score per cluster

write.csv(immune_percentage, "immune_cell_percentage.csv", row.names = FALSE)


In [ ]:
read.csv('/kaggle/working/Cluster_1_variable_genes.csv')

In [ ]:
library(dplyr)
library(Seurat)
library(ggplot2)
library(pheatmap)

# List of samples
samples <- c("A1", "B1", "C1", "D1")

# Compute immune scores
for (sample in samples) {
  st[[sample]]$Immune_Score <- PercentageFeatureSet(object = st[[sample]], pattern = "^IG")
}


In [ ]:

# List of sample names
samples <- c("A1", "B1", "C1", "D1")

# Initialize empty list to store metadata
immune_data_list <- list()

# Loop through each sample
for (sample in samples) {
  # Extract metadata
  meta_data <- st[[sample]]@meta.data
  
  # Add sample ID as a new column
  meta_data$Tissue <- sample
  
  # Store in list
  immune_data_list[[sample]] <- meta_data
}

# Combine all samples into one data frame
immune_metadata <- bind_rows(immune_data_list)

# Compute immune cell percentage per cluster per sample
immune_percentage <- immune_metadata %>%
  group_by(Tissue, clusters) %>%  # Adjust column name if different
  summarise(immune_cell_percent = mean(Immune_Score, na.rm = TRUE))

# Save as CSV
write.csv(immune_percentage, "immune_cell_percentage.csv", row.names = FALSE)


In [ ]:
head(immune_percentage)

In [ ]:
# Convert to wide format for heatmap
immune_matrix <- immune_percentage %>%
  pivot_wider(names_from = Tissue, values_from = immune_cell_percent) %>%
  column_to_rownames(var = "clusters") %>%  # Set clusters as row names
  as.matrix()
stage_annotation <- data.frame(
  Stage = c("IIIC/T2N3aM0", "IIA/T2N0M0", "I/T1cN0M0", "IIB/T3N0M0"),  # Avoid slashes
  row.names = c("A1", "B1", "C1", "D1")
)
# Make heatmap
p=pheatmap(immune_matrix,
             color = colorRampPalette(c("white", "red"))(100),  # Red color for high immune % 
                cluster_rows = TRUE, 
                cluster_cols = TRUE, 
                display_numbers = FALSE,
                main = "Immune Cell Percentage per Cluster per Sample",
             fontsize = 20,          # Increase overall font size
                fontsize_row = 20,      # Increase row label size
                fontsize_col = 20,      # Increase column label size
                legend = TRUE,
         annotation_col = stage_annotation,  # Add stage annotations
) 



ggsave("Heatmap.png", plot = p$gtable, width = 8, height = 6, dpi = 300)


In [ ]:
library(gridExtra)

# Save the heatmap correctly
png("Heatmap.png", width = 8, height = 6, units = "in", res = 300)
grid.draw(p$gtable)
dev.off()


In [ ]:
p

In [ ]:
# Define stage annotations manually
stage_labels <- c("A1_IIIC", "B1_IIA", "C1_I", "D1_IIB")

# Assign modified column names to the matrix
colnames(immune_matrix) <- stage_labels

# Generate heatmap
pheatmap(immune_matrix,
              color = colorRampPalette(c("white", "red"))(100),  
              cluster_rows = FALSE, 
              cluster_cols = FALSE, 
              display_numbers = FALSE,
              main = "Immune Cell Percentage per Cluster per Sample",
              fontsize = 14,          
              fontsize_row = 12,      
              fontsize_col = 30,      
              legend = TRUE)
# ggsave("Heatmap.png", plot=p, width=8, height=6, dpi=300)

In [ ]:
immune_matrix